In [35]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime  
from datetime import timedelta


# Following code makes relevant tokens necessary to conduct TD ameritrade activities on behalf of my account
# Note refresh token is valid for 90 days. It is used to generate auth token which is only valid for 30 mins
def get_auth_token():
    refresh_token = 'sXtUVLkSa8gbLhTjgVRPJDkx0J2khZOwk6uN+q0KlnrLoNuz6Fnm893VKp76zqml0GWZ73K1Oler9JJYMNFxziNe58G+SQ5zLgqLdA3EXVO3FXIPSR74glOqGddJ2hKH9tXL1oWft5WCE6ZUpq/zjPhsJSBPi8yGx2RnSABUc7FMqTeYOUcBztjjCUl4/jgk2F076hyrrDCSBX/yakGHA09jHcbQZOOl8vyrxu/UfrjEgfxNi4YuEMKz23pcHd2L77dJRBMTugYYnFzRpHMHdGukWedlgOO4/RgQgsZ3nAKEFrSYUS9XrugrREalD2Ufo+onBbkr+N1EP+BEyinufwOkrwsebgw/5FQKB9fJzbNw/79nXeS0ZG4OHlHA3shPP6JbJOUeh2Fwev/yuhrE6sGwfcdvftzn8vFTe52KGJrBe5RapEb+n+DxkJm100MQuG4LYrgoVi/JHHvlrueqQYnI53tYFD84wI4dnFH19sX584k0XZTqmBn4M2JDQqiO4SRIMAOVtkogDmlhOHMo1xDZVUG1aDRSdnoXe3EbnfnYI8h/PfIRyBUeVolmaUDfR5KUQ7etO7BMYlfC4w0hKhbcl1HayMZLi9lgtCPqW2cjoonhrywNt+TjlwG1zwxdfBL+T2YJmyb4jGYKe3r+/LfVV/Mo73zzodDajQtkCLjbtCcGPs8SrETi73eslhHuVxhUs73PBJBhwQAnwFXy1WsgNHANr3M733dxb/kJqlSkJTTN416SeuMuu69pM8j/T7AW1+Ly/d4OVjiIhpp5ZlU9U9YrSVdSQtD9A3Tt/VHI3jIHNn0yDMyweZlwfzzHKXDVEemNXPWXu3iGj/VZqujZ3RL0ZnpNHbQDf8PX+LjkY+0e7HteljkhgZlg4X0Q5F4YySluQCE=212FD3x19z9sWBHDJACbC00B75E'
    auth_params = {'grant_type':'refresh_token', 'refresh_token':'sXtUVLkSa8gbLhTjgVRPJDkx0J2khZOwk6uN+q0KlnrLoNuz6Fnm893VKp76zqml0GWZ73K1Oler9JJYMNFxziNe58G+SQ5zLgqLdA3EXVO3FXIPSR74glOqGddJ2hKH9tXL1oWft5WCE6ZUpq/zjPhsJSBPi8yGx2RnSABUc7FMqTeYOUcBztjjCUl4/jgk2F076hyrrDCSBX/yakGHA09jHcbQZOOl8vyrxu/UfrjEgfxNi4YuEMKz23pcHd2L77dJRBMTugYYnFzRpHMHdGukWedlgOO4/RgQgsZ3nAKEFrSYUS9XrugrREalD2Ufo+onBbkr+N1EP+BEyinufwOkrwsebgw/5FQKB9fJzbNw/79nXeS0ZG4OHlHA3shPP6JbJOUeh2Fwev/yuhrE6sGwfcdvftzn8vFTe52KGJrBe5RapEb+n+DxkJm100MQuG4LYrgoVi/JHHvlrueqQYnI53tYFD84wI4dnFH19sX584k0XZTqmBn4M2JDQqiO4SRIMAOVtkogDmlhOHMo1xDZVUG1aDRSdnoXe3EbnfnYI8h/PfIRyBUeVolmaUDfR5KUQ7etO7BMYlfC4w0hKhbcl1HayMZLi9lgtCPqW2cjoonhrywNt+TjlwG1zwxdfBL+T2YJmyb4jGYKe3r+/LfVV/Mo73zzodDajQtkCLjbtCcGPs8SrETi73eslhHuVxhUs73PBJBhwQAnwFXy1WsgNHANr3M733dxb/kJqlSkJTTN416SeuMuu69pM8j/T7AW1+Ly/d4OVjiIhpp5ZlU9U9YrSVdSQtD9A3Tt/VHI3jIHNn0yDMyweZlwfzzHKXDVEemNXPWXu3iGj/VZqujZ3RL0ZnpNHbQDf8PX+LjkY+0e7HteljkhgZlg4X0Q5F4YySluQCE=212FD3x19z9sWBHDJACbC00B75E', 'client_id':'TESTAUTH122@AMER.OAUTHAP'}
    auth_api_url = 'https://api.tdameritrade.com/v1/oauth2/token'
    return "Bearer "+ requests.post(auth_api_url, data=auth_params).json()['access_token']


# Following code will get the option chain data using the authorized token
# Parameters: call_or_put: 'CALL'/'PUT'; symbol: string with stock symbol from passed in stock list
def get_raw_option_data(call_or_put, symbol):
    url = 'https://api.tdameritrade.com/v1/marketdata/chains'
    strike_count = '500000' #need to figure out how to get ALL or just use an arbitraliy huge num
    strategy = 'SINGLE'
    authorization_header = {'Authorization': get_auth_token()}
    pay = {'symbol':symbol,'strikeCount':strike_count, 'strategy':strategy}
    if call_or_put == "CALL":
        return [requests.get(url, params = pay, headers = authorization_header).json()['callExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]
    elif call_or_put == "PUT":
        return [requests.get(url, params = pay, headers = authorization_header).json()['putExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]



# Following code does the filtering of the options data based on our chosen parameters
def processing(symbol, params, call_or_put):
    data = get_raw_option_data(call_or_put, symbol)
    agg_data = {}
    date_keys = []
    for (k,v) in data[0].items():
        date_keys.append([k,list(v.keys())])
        agg_data[k] = v
    print(agg_data[date_keys[0][0]][date_keys[0][1][0]][0]['putCall']) #This is an example of how we can access data. Can make less complicated if necessary using a different data structure but this should be good enough performace-wise
    target_list = []
    stock_price = data[1]
    if call_or_put == "CALL":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                if 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!
                    continue
                # construct a dictionary object characteristics we want and append to list
                target = {'symbol': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'],
                         '% OTM': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price,
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2}
                target_list.append(target)
    elif call_or_put == "PUT":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                if 1 + agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!      + or -???
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']) < params['cov_ret']: # Covered return Requirement! 
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*-2 < params['prob_touch']: # Probability of touching parameter      + or - ???
                    continue
                # construct a dictionary object characteristics we want and append to list
                target = {'symbol': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 + agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'], # + or - ???
                         '% OTM': stock_price/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2} # + or - ???
                target_list.append(target)
    df = pd.DataFrame(target_list)
    df = df.set_index('symbol', drop = True)
    return df



# Following function returns price data for a stock which can be used to construct a P&F chart
def return_candle_data(symbol):
    candle_url = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/pricehistory'
    candle_header = {'Authorization': get_auth_token()}
    # Vary the properties below depending on how often you want the data
    # Ideally, we want to set up a web socket to continuously stream this data?
    period_type = 'day'
    period = 2
    freq_type = 'minute'
    freq = 1
    candle_payload = {'periodType': period_type, 'period': period, 'frequencyType': freq_type, 'frequency': freq}
    raw_candle_data = requests.get(candle_url, params = candle_payload, headers = candle_header).json()
    df = pd.DataFrame(raw_candle_data['candles'])
    return df


# Get the relevant options after proving stock symbol and parameter list for filtering. 
def main():
    Stock_list = ['TSLA', 'AAPL'] # Need to populate this array with stock list we derive from technical indicators
    default_call_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15}
    charlie_call_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15}
    default_put_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15, 'cov_ret': 0.01, 'prob_touch': 0.01}
    charlie_put_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15, 'cov_ret': 0.01, 'prob_touch': 0.01}
    print(processing(Stock_list[0], default_call_params, "CALL"))
    
if __name__ == "__main__":
    main()



CALL
                      % OTM   ask   bid   cov_ret  last  max_cov_ret  \
symbol                                                                 
TSLA_071219C262.5  0.151442  0.96  0.89  6.457635  1.05     6.457635   
TSLA_071219C265    0.162408  0.81  0.75  6.707898  0.79     6.707898   
TSLA_071219C267.5  0.173374  0.68  0.62  6.978161  0.66     6.978161   
TSLA_071219C270    0.184340  0.57  0.52  7.268424  0.53     7.268424   
TSLA_071219C275    0.206273  0.41  0.38  7.928951  0.40     7.928951   
TSLA_071219C280    0.228205  0.30  0.26  8.609477  0.30     8.609477   
TSLA_071919C265    0.162408  1.56  1.47  5.006999  1.55     5.006999   
TSLA_071919C270    0.184340  1.15  1.07  5.067897  1.13     5.067897   
TSLA_071919C275    0.206273  0.86  0.79  5.258794  0.83     5.258794   
TSLA_071919C280    0.228205  0.64  0.59  5.519692  0.63     5.519692   
TSLA_071919C285    0.250137  0.49  0.44  5.840590  0.48     5.840590   
TSLA_071919C290    0.272069  0.38  0.34  6.201488  0.36    